In [1]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

In [2]:
data = pd.read_csv('lili.csv', sep=',',engine='python')
df = data

In [3]:
# remove nan row
isnan = df['skin'].apply(np.isnan)
notnan = np.invert(isnan)
index = df['skin'].index[notnan]
data_new = data.iloc[index]
# get features and thermal sensation
y = data_new['sensation']
x = data_new[['temperature','humidity','skin','clothing']]
x.to_csv('lili_feature.csv')

In [4]:
import sklearn.preprocessing, sklearn.decomposition,sklearn.linear_model, sklearn.pipeline, sklearn.metrics
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn import datasets, svm
from sklearn.model_selection import KFold, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn_pandas import DataFrameMapper

In [5]:
mapper = DataFrameMapper([(['temperature'], None),
                         (['humidity'], None),
                          (['skin'], None),
                          (['clothing'], None)])
mapper.fit_transform(x.copy())
# count the number of thermal sensation
bool = (y==3)
len(y[bool])

8

In [6]:
#clf = svm.SVC(kernel='linear')
C=1
#clf = svm.SVC(kernel='poly',degree=3,C=C)
clf = svm.SVC(kernel='rbf',gamma=0.7,C=C)
pipe = sklearn.pipeline.Pipeline([('featurize', mapper),('svc', clf)])
#np.round(cross_val_score(pipe, X=data_new.copy(), y=data_new.comfort, scoring='r2'), 2)
cross_val_score(pipe, X=x.copy(), y=y, scoring='r2',cv=5)

/Users/lusiliang93/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


array([ 0.38015464,  0.7254902 ,  0.64527027,  0.61794501,  0.67128874])

In [7]:
# testing
from sklearn.model_selection import cross_val_predict
from sklearn import metrics

In [8]:
predicted = cross_val_predict(clf, x.copy(), y, cv=5)
metrics.accuracy_score(y, predicted) 

/Users/lusiliang93/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


0.62209302325581395

In [9]:
test = x.iloc[[0]]
clf.fit(x,y)
gt = y[0]
pred = clf.predict(test)
pred
gt
test

,temperature,humidity,skin,clothing
0,-1.785714,-0.457604,-2.04924,21.7


In [10]:
import pickle
filename = 'svm_model.sav'
pickle.dump(clf,open(filename,'wb'),protocol=2)
load_model = pickle.load(open(filename,'rb'))
pred = load_model.predict(test)
pred

array([-1])